# U-Net Segmentation

A more sophisticated approach is needed to segment the wires from the images. Multiple ones are available but the first one used is the U-Net. 



Use pytorch to define the unet model 

In [1]:
# import modules (download with pip install first if not on local. Type on terminal: pip install <module name>)
import torch
import torch.nn as nn
import torch.nn.functional as F #contains some useful functions like activation functions & convolution operations you can use
import numpy as np

device = torch.device("cuda: 0" if torch.cuda.is_available() else "cpu")
print("Using",device,"...")

Using cuda:0 ...


In [2]:
# install torchvision first
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models

In [3]:
from load_data import *
# X, y = load_data('/content/drive/My Drive/AIWIRE', 'dataset')
X, y = load_data('.', 'dataset')

In [4]:
from tqdm import tqdm

# implement Simulation of Dataset 
class SimDataset(Dataset):
    def __init__(self, count,mode,transform=None):

# Simulate from matlab engine 
#        for i in tqdm(range(count)):
#            ground_truth, simulated = eng.simulate(1.3, 15, 0, nargout=2)
#            _simulated += [simulated]
#            _ground_truth += [ground_truth]
        if mode == '_train': 
          _simulated = X[0:int(count*X.shape[0])]
          _ground_truth = y[0:int(count*y.shape[0])]
        elif mode == '_val': 
          _simulated = X[-(int(count*X.shape[0])):]
          _ground_truth = y[-(int(count*y.shape[0])):]

        self.input_images, self.target_masks = np.array(_simulated), np.array(_ground_truth)
        self.transform = transform
    
    def __len__(self):
        return len(self.input_images)
    
    def __getitem__(self, idx):        
        image = self.input_images[idx]
        mask = self.target_masks[idx]
        if self.transform:
            image = self.transform(image)
            
        return [image, mask]

# Tranform into pytorch tensors 
trans = transforms.Compose([
    transforms.ToTensor(),
])

# Create a train set and a validation set, each with input images (simulation data) and target masks (ground truth data)
train_set = SimDataset(0.8,'_train',transform = trans)
val_set = SimDataset(0.2,'_val',transform = trans)

image_datasets = {
    'train': train_set, 'val': val_set
}

batch_size = 100

dataloaders = {
    'train': DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0),
    'val': DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0)
}

dataset_sizes = {
    x: len(image_datasets[x]) for x in image_datasets.keys()
}

dataset_sizes

{'train': 4000, 'val': 1000}

In [5]:
# Get a batch of training data
inputs, masks = next(iter(dataloaders['train']))

print(inputs.shape, masks.shape)
print(inputs.dtype, masks.dtype)

for x in [inputs.numpy(), masks.numpy()]:
    print(x.min(), x.max(), x.mean(), x.std())

# install torchsummary first
from torchsummary import summary
# import model from python file
import Unet_pytorch

model = Unet_pytorch.UNet(1)
model = model.to(device)

print("MODEL ARCHITECTURE ...")
summary(model, input_size=(1,192,128))

# Import other useful libraries 
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

loss_function = nn.MSELoss()

def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])
                    
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            epoch_samples = 0
            loss_vec = []

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)             

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    loss = loss_function(torch.squeeze(outputs), labels.type(torch.float32))
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        #print (loss.item())
                    loss_vec.append(loss.item())  

                    if epoch % 50 == 0:
                        print(loss.item())

                # statistics
                epoch_samples += inputs.size(0)

            # print loss at every epoch 
            print(np.asarray(loss_vec))
            epoch_loss = np.mean(np.asarray(loss_vec), dtype=np.float32)
            #epoch_loss = np.mean(np.asarray(loss_vec), dtype=np.float32)/float(epoch_samples)
            print('Loss ' + phase, ': {:.4f}'.format(epoch_loss))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

num_class = 1

model = Unet_pytorch.UNet(num_class).float()
model = model.to(device)

optimizer_ft = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9) 

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=25, gamma=0.1)

model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=40)

torch.Size([100, 1, 192, 128]) torch.Size([100, 192, 128])
torch.float64 torch.float64
9.822084046327182e-07 0.033971910984036915 0.003998812810005199 0.003363975758811057
0.0 1.0 0.010246988932291666 0.10070743840508088
MODEL ARCHITECTURE ...
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 192, 128]             320
              ReLU-2         [-1, 32, 192, 128]               0
            Conv2d-3         [-1, 32, 192, 128]           9,248
              ReLU-4         [-1, 32, 192, 128]               0
         MaxPool2d-5           [-1, 32, 96, 64]               0
            Conv2d-6           [-1, 64, 96, 64]          18,496
              ReLU-7           [-1, 64, 96, 64]               0
            Conv2d-8           [-1, 64, 96, 64]          36,928
              ReLU-9           [-1, 64, 96, 64]               0
        MaxPool2d-10           [-1, 64, 48, 32]    

/vol/bitbucket/rh2515/CrohnsDisease/venv/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


0.01893678680062294
0.018416741862893105
0.01750318519771099
0.016345517709851265
0.015116836875677109
0.01388542540371418
0.01270214281976223
0.011608261615037918
0.010967533104121685
0.010452117770910263
0.0101993503049016
0.01009098906069994
0.010259639471769333
0.010397220030426979
0.010685240849852562
0.010883726179599762
0.011202745139598846
0.011247883550822735
0.011350240558385849
0.01126311905682087
0.011245938017964363
0.01104987133294344
0.011004699394106865
0.010866646654903889
0.0106572974473238
0.010448667220771313


KeyboardInterrupt: 